In [ ]:
!mkdir Dataset
!mkdir Dataset/Extent
!wget  https://zenodo.org/records/12723317/files/StudyArea.gpkg?download=1  -O Dataset/Extent/StudyArea.gpkg

In [ ]:
import geopandas as gpd
import openeo
import json

In [ ]:
connection = openeo.connect(url="openeo.dataspace.copernicus.eu").authenticate_oidc()

In [ ]:
aoi = json.loads(gpd.read_file("Dataset/Extent/StudyArea.gpkg").to_json())

In [ ]:
sentinel2_cube = connection.load_collection(
    "SENTINEL2_L2A",
    spatial_extent=aoi,
    temporal_extent=["2019-10-13", "2019-11-05"],
    bands=["B02", "B03", "B04", "B08"],
)

In [ ]:
sentinel1_pre_cube = connection.load_collection(
    "SENTINEL1_GRD",
    spatial_extent=aoi,
    temporal_extent=["2019-09-01", "2019-10-01"],
    bands=["VV"],
    properties={"sat:orbit_state": lambda v: v == "ascending"},
)
sentinel1_pre_cube = sentinel1_pre_cube.sar_backscatter(coefficient="gamma0-ellipsoid")

In [ ]:
sentinel1_post_cube = connection.load_collection(
    "SENTINEL1_GRD",
    spatial_extent=aoi,
    temporal_extent=["2019-10-13", "2019-11-05"],
    bands=["VV"],
    properties={"sat:orbit_state": lambda v: v == "ascending"},
)
sentinel1_post_cube = sentinel1_post_cube.sar_backscatter(
    coefficient="gamma0-ellipsoid"
)

In [ ]:
sentinel2_cube = sentinel2_cube.reduce_dimension(dimension="t", reducer="median")

In [ ]:
sentinel1_pre_cube = sentinel1_pre_cube.reduce_dimension(dimension="t", reducer="last")

In [ ]:
sentinel1_post_cube = sentinel1_post_cube.reduce_dimension(
    dimension="t", reducer="first"
)

In [ ]:
ratio = sentinel1_post_cube.divide(sentinel1_pre_cube)

# rename for merging
ratio = ratio.rename_labels(dimension="bands", target=["ratio"])
sentinel1_post_cube = sentinel1_post_cube.rename_labels(
    dimension="bands", target=["postSAR"]
)
sentinel1_pre_cube = sentinel1_pre_cube.rename_labels(
    dimension="bands", target=["preSAR"]
)

In [ ]:
radar_cubes = [sentinel1_pre_cube, sentinel1_post_cube, ratio]
merged_cube = sentinel2_cube
for cube in radar_cubes:
    merged_cube = merged_cube.merge_cubes(cube)

In [ ]:
job_options = {
    "executor-memory": "3G",
    "executor-memoryOverhead": "4G",
    "executor-cores": "2",
}
merged_cube.execute_batch(
    title="Download Japan", outputfile="Data/JapanImages.tiff", job_options=job_options
)